# Exercise: Cheating input/output layouts

The output layout of a GUfunc has to be in some way a combination of the input layouts.  That is, the input/output layout

```python
(m),(n)->(r)
```

doesn't work.  But sometimes that's what needs to happen, so how to get around this limitation?  

Try it out!

Uncomment the `guvectorize` decorator, add a signature and make whatever changes you need to to make it work

In [ ]:
import numpy

In [ ]:
from numba import guvectorize

In [ ]:
#@guvectorize
def return_half_array(in_array, out_array):
    for i in range(in_array.shape[0] // 2):
        out_array[i] = in_array[i]

In [ ]:
in_array = numpy.arange(20)
out_array = numpy.empty(10)

In [ ]:
return_half_array(in_array, out_array)

# Exercise: Multigrid Restriction

Multigrid is a method used to solve linear systems quickly.  I'm going to be very light on details here, but suffice it to say that one of the major components of this involves repeatedly coarsening and interpolating values from finer to coarser grids and back.  

Consider the figure below and how we can populate a coarse grid using a weighted average of values from the fine grid.

<img src='../figures/2d_full_weighting.svg' width=600>

<img src='../figures/2d_full_weighting_detail.svg' width=600>

In 2D, the full weighting restriction operator is given as

\begin{align}
v_{(i,j),c} &=\frac{1}{16} \left(v_{(2i-1,2j-1),f} +v_{(2i-1,2j+1),f} +v_{(2i+1,2j-1),f}  +v_{(2i+1,2j+1),f} \right)\notag \\
&\ + \frac{1}{8} \left(v_{(2i,2j-1),f} + v_{(2i,2j+1),f} +v_{(2i-1,2j),f}  +v_{(2i+1,2j),f} \right) \\
&\ + \frac{1}{4} v_{(2i,2j),f} \\
& \texttt{ for } \left\lbrace 1 \le i,j \le N_{x,c}-2 \right.
\end{align}

where $v_c$ is the coarse grid and $v_f$ is the fine grid

In [ ]:
def restrict_2d_gvec():

Want to check your answer? If you use

In [ ]:
fine = numpy.arange(100).reshape(10,10)
coarse = numpy.zeros((5, 5))

You should end up with 

```
array([[  0.  ,   0.  ,   0.  ,   0.  ,   0.  ],
       [  0.  ,  19.25,  21.  ,  22.75,   0.  ],
       [  0.  ,  36.75,  38.5 ,  40.25,   0.  ],
       [  0.  ,  54.25,  56.  ,  57.75,   0.  ],
       [  0.  ,   0.  ,   0.  ,   0.  ,   0.  ]])
       ```

# Exercise: Multigrid Interpolation

We can go from fine to coarse -- now to go from coarse to fine.  

How does that work?  More weighted averages.  In the figure below, the circles represent values on the coarse array, squares and triangles are interpolated values on the fine array.  The arrows indicate the weighting to use in calculating the fine array values.

<img src="../figures/2d_interpolation.svg" width=500>

The **interpolation in 2D** is

\begin{align}
v_{(2i,2j),f} &= v_{(i,j),c} \texttt{ for } \left\{ 0 \le i,j \le N_{x,c}-2 \right.\\
v_{(2i+1,2j),f} &= \frac{1}{2}\left(v_{(i,j),c}+v_{(i+1,j),c} \right) 
\texttt{ for }  \left\lbrace 0 \le i,j \le N_{x,c}-2 \right .\\
v_{(2i,2j+1),f} &= \frac{1}{2}\left(v_{(i,j),c}+v_{(i,j+1),c} \right) \texttt{ for } \left\lbrace 0 \le i,j \le N_{x,c}-2 \right.\\
v_{(2i+1,2j+1),f} &= \frac{1}{4}\left(v_{(i,j),c}+v_{(i+1,j),c}+v_{(i,j+1),c}+v_{(i+1,j+1),c}\right) \texttt{ for } \left\lbrace 0 \le i,j \le N_{x,c}-2 \right.
\end{align}